In [ ]:
''' upload to hub
from parallel_processing import deeplake_driver as dl_driver
# import deeplake as dl

BATCH_NAME             = 'parallel_15'
# RESULTS_DATASET_PATH   = f'/mnt/storage_ssd/clip_OUTPUT_results_{BATCH_NAME}'
RESULTS_DATASET_PATH   = f'~/VPT/v4_CLIP_encode_results_{BATCH_NAME}' # overdrive

dl_driver.upload_dataset_to_hub(RESULTS_DATASET_PATH)
'''

In [2]:
import deeplake as dl
import tqdm
BATCH_NAME             = 'parallel_15'
# new_name   = f'/mnt/storage_ssd/dec_23_clip_results_{BATCH_NAME}'
# new_name   = f'/mnt/storage_ssd/v3_text_encode_results_{BATCH_NAME}'
# new_name   = f'/mnt/storage_hdd/thesis/yt_1b_dataset/backups/v3_text_encode_results_{BATCH_N/AME}'
# new_name   = f'/mnt/storage_hdd/thesis/yt_1b_dataset/backups/no_compression_whisper_results_{BATCH_NAME}'
# new_name   = f'/mnt/storage_hdd/thesis/yt_1b_dataset/backups/v3_text_encode_results_{BATCH_NAME}'
# new_name   = f'/mnt/storage_ssd/dec_26_whisper_results_parallel_15'
RESULTS_DATASET_PATH   = f'~/VPT/v4_CLIP_encode_results_{BATCH_NAME}' # overdrive
# RESULTS_DATASET_PATH = f'/mnt/storage_ssd/v3_CLIP_encode_results_{BATCH_NAME}'  # ssd
ds = dl.load(RESULTS_DATASET_PATH)
print(ds.size_approx() / 1e9, "GB")
ds.summary()

ModuleNotFoundError: No module named 'pandas'

In [17]:
import json
json.loads(ds.segment_metadata[0].data()['value'])

{'segment_index': 0,
 'total_segments': 490,
 'captions': 'HELLO EVERYBODY AND WELCOME BACK TO MY SECOND CHANNEL I KNOW IT HAS BEEN A',
 'start': 0.320875,
 'end': 3.8108750000000002}

In [9]:
for idx, sample in tqdm.tqdm(enumerate(ds)):
    # sample['text'] = sample['text'].decode('utf-8')
    metadata = sample['segment_metadata'].data()['value']
    
    if metadata:
      print(metadata)
      print("found something ^^^^^^^^^^")
      break
    # if idx >= 500:
      # break

506889it [01:06, 7595.95it/s] 


KeyboardInterrupt: 

In [ ]:



import deeplake as dl
import tqdm
dataset_name = '/mnt/storage_ssd/v2_text_encode_results_parallel_15'
# dataset_name = '/mnt/storage_ssd/v2_whisper_results_parallel_15'
# dataset_name = '/mnt/storage_ssd/problem_file_only_whisper_results_parallel_15'
ds = deeplake.load(dataset_name)
print(ds.size_approx() / 1e9, "GB")
ds.summary()

In [ ]:
dataset_name = '/mnt/storage_ssd/text_encode_results_parallel_15'
ds = deeplake.load(dataset_name)
print(ds.size_approx() / 1e9, "GB")
ds.summary()
print(type(ds.caption_embedding[0].data()['value']))
print(ds.caption_embedding[0].data()['value'].shape)

In [ ]:
# make a backup
# deeplake.deepcopy(dataset_name_v2, '/mnt/storage_ssd/BACKUP_no_compression_whisper_results_parallel_15')

In [ ]:
# new_ds[19422].caption_embedding.numpy()
new_ds.summary()

In [ ]:
''' CONVERT dataset without compression'''
import numpy as np
old_ds_path = '/mnt/storage_ssd/text_encode_results_parallel_15'
old_ds = deeplake.load(old_ds_path)

new_ds_path = '/mnt/storage_ssd/fresh_text_encode_parallel_15'
new_ds = deeplake.empty(new_ds_path, overwrite=True)

with new_ds:
  new_ds.create_tensor('caption', htype='text', dtype=str, sample_compression=None) # todo: change to chunk_compression
  new_ds.create_tensor('video_filename', htype='text', dtype=str, sample_compression=None)
  new_ds.create_tensor('video_filepath', htype='text', dtype=str, sample_compression=None)
  new_ds.create_tensor('segment_metadata', htype='json', sample_compression=None)
  new_ds.create_tensor('caption_embedding', htype='image', dtype=np.float16, sample_compression='lz4')

  for idx, sample in tqdm.tqdm(enumerate(old_ds)):
    try:
      new_ds.caption.append( sample.caption.data()['value']   )
      new_ds.video_filename.append( sample.video_filename.data()['value']   )
      new_ds.video_filepath.append( sample.video_filepath.data()['value']   )
      new_ds.segment_metadata.append( sample.segment_metadata.data()['value']   )
      new_ds.caption_embedding.append( sample.caption_embedding.data()['value']   )
    except Exception as e:
      print("error at idx:", idx, e)
new_ds.summary()

In [ ]:
ds.tensors.keys()

In [ ]:
dataset_name = '/mnt/storage_ssd/v2_whisper_results_parallel_15'
ds_copy = deeplake.empty(dataset_name, overwrite=True)
ds = ds_copy_var_only
ds.flush()
ds.size_approx()

In [ ]:
dataset_name = '/mnt/storage_ssd/whisper_results_parallel_15'
loaded = deeplake.load(dataset_name)
loaded.summary()

In [ ]:
# deeplake.copy(ds_copy, '/mnt/storage_ssd/v3_whisper_results_parallel_15', overwrite=True)

In [ ]:
# jus looking
ds_copy_var_only = ds[:16667]
ds_copy_var_only.summary()

# metadata = sample.segment_metadata.data()['value']
metadata = ds_copy_var_only[16667].segment_metadata.data()['value']
print(metadata['segment_index'], "of", metadata['total_segments'], "total")

In [ ]:
# delete bad tensors

last_good_index = 19183
with new_ds: 
  for i in range((new_ds.max_len-1), last_good_index, -1):
    print(i)
    new_ds.caption.pop(i)
    new_ds.video_filename.pop(i)
    new_ds.video_filepath.pop(i)
    new_ds.segment_metadata.pop(i)
    try:
      new_ds.caption_embedding.pop(i)
    except Exception as e:
      print(e)
    

In [ ]:
ds.summary()

In [ ]:
ds.flush()

In [ ]:
vds_path = ds[:16705].save_view(path="/mnt/storage_ssd/V3_ONLY_VALID_whisper_results_parallel_15", id="only_valid")
vds_path

In [ ]:
print(ds.summary())

import lovely_tensors as lt
lt.monkey_patch()

# index_caption_pairs = []
not_done = 0
with ds:
  for idx, sample in enumerate(ds):
    # if not done
    if not sample.caption_embedding.numpy().any():
      print(idx, sample.caption_embedding.numpy())
      not_done += 1
    
    # if sample.caption_embedding.numpy().shape == (0, 0):
    #   index_caption_pairs.append( {'db_index': idx, 'caption': sample.caption.data()['value']} )
# index_caption_pairs
print(not_done)

In [ ]:
new_ds = deeplake.empty('/mnt/storage_ssd/fresh_text_encode_parallel_15', overwrite=True)

for idx, sample in enumerate(ds):
  # find 
  try: 
    new_ds.caption_embedding.append( sample.caption_embedding.numpy() )
    # if sample.caption_embedding.numpy().shape == (0, 0):
    #   index_caption_pairs.append( {'db_index': idx, 'caption': sample.caption.data()['value']} )
  except IndexError as e:
    print(e)
    # if there's an IndexError, then the caption_embedding is empty.
    index_caption_pairs.append( {'db_index': idx, 'caption': sample.caption.data()['value']} )

In [ ]:
ds.caption_embedding[15214].numpy().shape

In [ ]:
# data access
import pprint
import traceback

indexes_with_no_pcation_embedding = []
for i in range(ds.max_len):
  try: 
    if ds.caption_embedding[i].numpy().shape != (0, 0):
      # print(i, "✅ of", ds.max_len)
      pass
    else:
      print(i, "❌ NONE of", ds.max_len, "shape: ", ds.caption_embedding[i].numpy().shape)
      indexes_with_no_pcation_embedding.append(i)
  except Exception as e:
    print(e)

In [ ]:
len(indexes_with_no_pcation_embedding)

In [ ]:
import deeplake as dl
import numpy as np
junk = deeplake.empty('/tmp/hi', overwrite=True)
junk.create_tensor('penis',   htype='image', dtype=np.float32, sample_compression=None)
for i in range(10):
  junk.penis.append(None)

junk.penis[5] = np.float32(69)
print(junk.summary())

for sample in junk:
  if sample.penis.numpy().shape == (0,):
    print("it was none")
  else:
    print(sample.penis.numpy().shape)

## Dataset management (filtering, uploading, checkpointing)

## Docs

Main reference
https://docs.deeplake.ai/en/latest/Datasets.html

In [ ]:
import deeplake
dataset_name = '/mnt/storage_ssd/whisper_results_parallel_15'
read_lake = deeplake.load(dataset_name)
print(read_lake.size_approx() / 1e9, "GB")
read_lake.summary()

In [ ]:
# take first n samples
read_lake = read_lake[:200]

In [ ]:
read_lake.summary()

In [ ]:
# data access
import pprint
for i, sample in enumerate(read_lake):
  pprint.pprint(sample.segment_metadata.data()['value'])
  pprint.pprint(sample.video_filepath.data()['value'])
  break

In [ ]:
# upload to hub
deeplake.copy(read_lake, dest="hub://kastan/p17_test")

In [ ]:
import os
read_lake.add_creds_key('lBEly4ezopu3j54yvdehHYtLPQz6LJBzVx3Pmxpi')
# print(os.environ.get('KAS_S3_KEY'))

In [ ]:
# s3_bucket = deeplake.empty("s3://vpt-deeplake/p17_test")
# upload to s3
deeplake.copy(read_lake, dest="s3://vpt-deeplake/final_clip_p15")
## Todo: parallel. num_workers=8, scheduler='processed' or 'threaded' or 'ray'

In [ ]:
# all_stems = set(read_lake.video_stem.data()['value'])
# print(len(all_stems))
# read_lake.query("select * where contains(video_stem, 'car')")

def filter_func(sample):
  # print(sample.segment_metadata.data()['value'])
  caption = sample.segment_metadata.data()['value']['captions']
  if 'computer'.upper() in caption.upper():
    # print("returning true")
    return True 
  elif 'kiss'.upper() in caption.upper():
    return True
  else:
    # print('false')
    return False

filtered_dataset = read_lake.filter(filter_func, num_workers = 2, scheduler = 'threaded')
# read_lake.filter("'computer' in segment_metadata.data()['value']['captions']", num_workers = 10, scheduler = 'ray')

In [ ]:
import pprint
filtered_dataset.summary()
for i, sample in enumerate(filtered_dataset):
  pprint.pprint(sample.segment_metadata.data()['value'])
  
  if i > 20: 
    break

In [ ]:
# read_lake.segment_metadata[2].data()['value'][total_segments]
# read_lake.text_caption_embeddings[2].data()['value']

read_lake = filtered_dataset

import cv2
from PIL import Image
for i in range(110):
  # frame = read_lake.segment_frames[i].numpy()
  # frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
  display(Image.fromarray(read_lake.segment_frames[i].numpy()))
  print(read_lake.segment_metadata[i].data()['value']['captions'])
  metadata = read_lake.segment_metadata[i].data()['value']
  print(metadata['segment_index'], "of", metadata['total_segments'])
  